In [20]:
%env CUDA_VISIBLE_DEVICES = 1

env: CUDA_VISIBLE_DEVICES=1


In [21]:
%load_ext autoreload
%autoreload 2
import sys; 
sys.path.extend(['/home/meet/FlowMatchingTests/conditional-flow-matching/'])

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchcfm.conditional_flow_matching import *
from physics_flow_matching.unet import UNetModel
from torchdiffeq import odeint
from einops import rearrange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# data_0_150 = np.load('/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y5-150_41-220_out.npy')

In [23]:
data = [np.load('/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_all_out.npy')] #, data_0_150[:, :1]
for i in [5, 10, 40, 60, 70, 80, 100, 150]:
    data.append(np.load(f'/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y{i}_all_out.npy'))
data = np.concatenate(data, axis=1)

In [24]:
m_, std_ = data[:1200].mean(axis=(0,2,3), keepdims=True), data[:1200].std(axis=(0,2,3), keepdims=True)

In [25]:
# m_, std_  = m_[:, [0, 1, 4, 6, 7, 8]], std_[:, [0, 1, 4, 6, 7, 8]]

In [26]:
# test_data_0_150 = np.load('/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y5-150_221-259_out.npy')

In [27]:
# test_data = [np.load('/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_wall_dudy_221-259_out.npy')[:, None], test_data_0_150[:, :1]]
# for i in ["60-80-100"]:
#     test_data.append(np.load(f'/home/xiantao/case/wall_pressure/bigchannel/data/to_meet/channel_180_u_y{i}_221-259_out.npy'))
# test_data.append(test_data_0_150[:, 1:])
# test_data = np.concatenate(test_data, axis=1)

In [28]:
X = (data[36000:39900] - m_)/std_

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
exp=4
iteration = 19
ot_cfm_model = UNetModel(dim=[1, 160, 200],
                         num_channels=128,
                         out_channels=1,
                         num_res_blocks=2,
                         num_head_channels=64,
                         attention_resolutions="40, 20",
                         dropout=0.0,
                         use_new_attention_order=True,
                         use_scale_shift_norm=True)
state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wm_ar/exps/exp_{exp}/saved_state/checkpoint_{iteration}.pth")
ot_cfm_model.load_state_dict(state["model_state_dict"])
ot_cfm_model.to(device)
ot_cfm_model.eval();

/tmp/ipykernel_3389016/3171781994.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_fl

In [31]:
batch_size=10
ind = 3

In [32]:
samples_list = []
for batch in range(X.shape[0]//batch_size):
    each_samples_list =[]
    with torch.no_grad():
        for i in range(8-ind):
            if i == 0:
                out = ot_cfm_model(torch.tensor(X[batch*batch_size:(batch+1)*batch_size, ind:ind+1], device=device)).detach().cpu().numpy()
            else:
                out = ot_cfm_model(torch.tensor(out, device=device)).detach().cpu().numpy()
            sout = out*std_[:, ind+i+1:ind+i+2]
            sout += m_[:, ind+i+1:ind+i+2]
            each_samples_list.append(sout)
    samples_list.append(each_samples_list)

In [33]:
# samples_list = []
# for batch in range(X.shape[0]//batch_size):
#     with torch.no_grad():
#         out = ot_cfm_model(torch.tensor(X[batch*batch_size:(batch+1)*batch_size, ind:ind+1], device=device)).detach().cpu().numpy()
#     out *= std_[:, ind+1:ind+2]
#     out += m_[:, ind+1:ind+2]
#     samples_list.append(out)

In [34]:
sample_list2 = [np.concatenate(sample, axis=1) for sample in samples_list]
samples = np.concatenate(sample_list2)

In [35]:
# np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/vf_vf/exps/exp_{exp}/samples_{iteration}epch", samples[:, 0])

In [36]:
# for i in range(8):
#     fig, axes = plt.subplots(nrows=1, ncols=4, dpi=200)
#     if i == 0:
#         ax = axes[0].imshow(data[36000,0])
#         plt.colorbar(ax,ax=axes[0])
#     else:
#         ax = axes[0].imshow(samples[0,i-1])
#         plt.colorbar(ax,ax=axes[0])
#     ax = axes[1].imshow(samples[0,i])
#     plt.colorbar(ax,ax=axes[1])
#     ax= axes[2].imshow(data[36000, i+1])
#     plt.colorbar(ax,ax=axes[2])
#     ax = axes[3].imshow(np.abs(samples[0,i] - data[36000, i+1]))
#     plt.colorbar(ax,ax=axes[3])

In [37]:
# for i in range(1):
#     fig, axes = plt.subplots(nrows=1, ncols=4, dpi=500, figsize=(20,10))
#     ax = axes[0].imshow(X[7800+i,ind])
#     plt.colorbar(ax,ax=axes[0])
#     ax = axes[1].imshow(samples[i,0])
#     plt.colorbar(ax,ax=axes[1])
#     ax= axes[2].imshow(data[7800+i, ind+1])
#     plt.colorbar(ax,ax=axes[2])
#     ax = axes[3].imshow(np.abs(samples[i,0] - data[7800+i, ind+1]))
#     plt.colorbar(ax,ax=axes[3])

In [38]:
np.save(f"/home/meet/FlowMatchingTests/conditional-flow-matching/physics_flow_matching/wm_ar/exps/exp_{exp}/samples_{iteration}iter_7000_start_40", samples)